In [1]:
import os

import nibabel as nb
import numpy as np
from functions import convert_samples_to_features, edit_events_musicnoise, extract_samples
from nilearn import image

In [2]:
# data_root = "/Volumes/T7/BIDS-BRAINPLAYBACK-TASK2"
data_root = "/DATAPOOL/BRAINPLAYBACK/BIDS-BRAINPLAYBACK-TASK2"  # sim01 dir

derivatives_dir = os.path.join(data_root, "derivatives")

fmriprep_dir = os.path.join(derivatives_dir, "fmriprep23")  # fmriprep output
func_dir = os.path.join(derivatives_dir, "func_clean")  # cleaned bold images
stab_mask_dir = os.path.join(derivatives_dir, "mvpa_08_stability_bold", "stab_musicnoise_masks")  # for stability masks
output_feat_full_dir = os.path.join(derivatives_dir, "mvpa_08_stability_bold", "features_musicnoise")  # for full features

In [3]:
# check if output directories exist and create them if not
if not os.path.exists(output_feat_full_dir):
    os.makedirs(output_feat_full_dir)

# All subject iteration

In [4]:
from concurrent.futures import ThreadPoolExecutor

subject_list = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"]
run_list = ["1", "2", "3", "4"]


def process_subject(subject):
    print(f"Processing subject {subject}")
    stab_mask_data = np.load(os.path.join(stab_mask_dir, f"sub-{subject}_STAB.npy"))
    example_func = nb.load(os.path.join(func_dir, f"sub-{subject}_ses-01_task-02a_run-1_cleaned.nii.gz"))

    # generate image
    stab_mask_img = nb.Nifti1Image(stab_mask_data, example_func.affine, example_func.header)

    # threshold the stability map
    thres = 0.1
    stab_mask_img_thres = image.threshold_img(stab_mask_img, threshold=thres, two_sided=False)
    stab_mask_img_thres_bin = image.binarize_img(stab_mask_img_thres)

    # save the thresholded stability mask
    stab_mask_img_thres_bin.to_filename(os.path.join(stab_mask_dir, f"sub-{subject}_STAB_mask.nii.gz"))

    for run in run_list:
        if subject == "18" and run == "2":
            continue

        # load cleaned image
        img_clean = nb.load(os.path.join(func_dir, f"sub-{subject}_ses-01_task-02a_run-{run}_cleaned.nii.gz"))

        # extract samples from the cleaned inside the stability mask
        samples = extract_samples(img_clean, stab_mask_img_thres_bin, subject, run)

        # load and edit the events for classification
        events_split_2 = edit_events_musicnoise(data_root, subject, run)

        # convert samples to features
        convert_samples_to_features(samples, output_feat_full_dir, events_split_2, subject, run)


with ThreadPoolExecutor(max_workers=18) as executor:
    executor.map(process_subject, subject_list)

Processing subject 01
Processing subject 02
Processing subject 03
Processing subject 04
Processing subject 05
Processing subject 06
Processing subject 07
Processing subject 08
Processing subject 09
Processing subject 10
Processing subject 11
Processing subject 12
Processing subject 13
Processing subject 14
Processing subject 15
Processing subject 16


/tmp/ipykernel_833884/1105022159.py:17: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  stab_mask_img_thres = image.threshold_img(stab_mask_img, threshold=thres, two_sided=False)


Processing subject 17
Processing subject 18


/tmp/ipykernel_833884/1105022159.py:18: DeprecationWarning: The current default behavior for the "two_sided" argument is  "True". This behavior will be changed to "False" in version 0.13.
  stab_mask_img_thres_bin = image.binarize_img(stab_mask_img_thres)
/tmp/ipykernel_833884/1105022159.py:18: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  stab_mask_img_thres_bin = image.binarize_img(stab_mask_img_thres)


Extracting samples for subject 02, run 1...
Extracting samples for subject 01, run 1...
Extracting samples for subject 10, run 1...
Extracting samples for subject 03, run 1...
Extracting samples for subject 14, run 1...
Extracting samples for subject 05, run 1...
Extracting samples for subject 15, run 1...
Extracting samples for subject 07, run 1...
Extracting samples for subject 09, run 1...
Extracting samples for subject 06, run 1...
Extracting samples for subject 08, run 1...
Extracting samples for subject 13, run 1...
Extracting samples for subject 11, run 1...
Extracting samples for subject 17, run 1...
Extracting samples for subject 18, run 1...
Extracting samples for subject 04, run 1...
Extracting samples for subject 16, run 1...
Extracting samples for subject 12, run 1...
Samples extracted for subject 09, run 1.
Editing events for subject 09, run 1...
Events edited for subject 09, run 1.
Converting samples to features for subject 09, run 1...
Samples extracted for subject 16, 

/tmp/ipykernel_833884/1105022159.py:17: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  stab_mask_img_thres = image.threshold_img(stab_mask_img, threshold=thres, two_sided=False)
/tmp/ipykernel_833884/1105022159.py:18: DeprecationWarning: The current default behavior for the "two_sided" argument is  "True". This behavior will be changed to "False" in version 0.13.
  stab_mask_img_thres_bin = image.binarize_img(stab_mask_img_thres)
/tmp/ipykernel_833884/1105022159.py:18: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  stab_mask_img_thres_bin = image.binarize_img(st

Features extracted and labels saved for subject 04, run 3.
Extracting samples for subject 04, run 4...
Extracting samples for subject 19, run 1...
Samples extracted for subject 17, run 3.
Editing events for subject 17, run 3...
Events edited for subject 17, run 3.
Converting samples to features for subject 17, run 3...
Features extracted and labels saved for subject 17, run 3.
Extracting samples for subject 17, run 4...
Samples extracted for subject 10, run 3.
Editing events for subject 10, run 3...
Events edited for subject 10, run 3.
Converting samples to features for subject 10, run 3...
Features extracted and labels saved for subject 10, run 3.
Extracting samples for subject 10, run 4...
Samples extracted for subject 02, run 3.
Editing events for subject 02, run 3...
Samples extracted for subject 16, run 3.
Editing events for subject 16, run 3...
Events edited for subject 02, run 3.
Converting samples to features for subject 02, run 3...
Events edited for subject 16, run 3.
Convert

/tmp/ipykernel_833884/1105022159.py:17: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  stab_mask_img_thres = image.threshold_img(stab_mask_img, threshold=thres, two_sided=False)
/tmp/ipykernel_833884/1105022159.py:18: DeprecationWarning: The current default behavior for the "two_sided" argument is  "True". This behavior will be changed to "False" in version 0.13.
  stab_mask_img_thres_bin = image.binarize_img(stab_mask_img_thres)
/tmp/ipykernel_833884/1105022159.py:18: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  stab_mask_img_thres_bin = image.binarize_img(st

Samples extracted for subject 09, run 4.
Editing events for subject 09, run 4...
Events edited for subject 09, run 4.
Converting samples to features for subject 09, run 4...
Features extracted and labels saved for subject 09, run 4.
Extracting samples for subject 20, run 1...
Samples extracted for subject 17, run 4.
Editing events for subject 17, run 4...
Events edited for subject 17, run 4.
Converting samples to features for subject 17, run 4...
Features extracted and labels saved for subject 17, run 4.
Samples extracted for subject 08, run 4.
Editing events for subject 08, run 4...
Events edited for subject 08, run 4.
Converting samples to features for subject 08, run 4...
Features extracted and labels saved for subject 08, run 4.
Samples extracted for subject 02, run 4.
Editing events for subject 02, run 4...
Events edited for subject 02, run 4.
Converting samples to features for subject 02, run 4...
Features extracted and labels saved for subject 02, run 4.
Samples extracted for su